# Laminar Time-Dependent Navier-Stokes Flow in Tank with Nozzle using Ferrite   

## Import Packages

In [3]:
using BlockArrays
using LinearAlgebra
using UnPack
using LinearSolve 
using SparseArrays
using Ferrite
using FerriteGmsh 
using OrdinaryDiffEq
using DifferentialEquations
using Plots 
using WriteVTK

## Section 1: Introduction 

### Goal

To simulate transient laminar flow in a cylindrical tank with nozzle using [Ferrite.jl](https://ferrite-fem.github.io/Ferrite.jl/stable/) and [DifferentialEquations.jl](https://docs.sciml.ai/DiffEqDocs/stable/) (in similar project this will become cylindrical cavity with periodically oscillating lid (or cover)). The mesh required on input is generated using [GMSH.jl](https://github.com/JuliaFEM/Gmsh.jl) (see seperate notebook). This code was modeled after the tutorial in Ferrite [Incompressible Navier-Stokes equations via DifferentialEquations.jl](https://ferrite-fem.github.io/Ferrite.jl/stable/tutorials/ns_vs_diffeq/). 

<img src="./laminar_stokes_scheme.png" width=800 />

#### Nomenclature

### Strong Forms of Governing Equations for Laminar Flow (2D)

We consider the following equations for laminar flow in 2D (x, z), assuming incompressible flow.

**Equation 1: Conservation of Mass for the Gaz Phase**

$$\nabla \cdot \mathbf{u} = 0$$

**Equation 2: Conservation of Momentum for the Gas Phase**

* *x-momentum equation:*
$$
\rho_g \left( \frac{\partial u_x}{\partial t} + u_x \frac{\partial u_x}{\partial x} + u_z \frac{\partial u_x}{\partial z} \right) = -\frac{\partial p}{\partial x} + \mu \left( \frac{\partial^2 u_x}{\partial x^2} + \frac{\partial^2 u_x}{\partial z^2} \right) - S_x
$$

* *z-momentum equation:*
$$
\rho_g \left( \frac{\partial u_z}{\partial t} + u_x \frac{\partial u_z}{\partial x} + u_z \frac{\partial u_z}{\partial z} \right) = -\frac{\partial p}{\partial z} + \mu \left( \frac{\partial^2 u_z}{\partial x^2} + \frac{\partial^2 u_z}{\partial z^2} \right) - S_z
$$

With $S_r$ and $S_z$ represent the pressure loss in the bed due to viscous dissipation. $S_i$ in each direction is then calculated by:

$$S_i = \frac{\mu}{K}u_i$$, 

Where K denotes the permeability of the porous bed, $K=10^{-9} m^2$

---

### Initial and Boundary conditions

#### Initial Conditions (IC)

* Initial Solid Density ($\rho_s(t=0)$):
    * *Absorption:* Initial solid density is equal to the density of the metal hydride bed without hydrogen ($\rho_{\text{emp}}$).
    * *Desorption:* Initial solid density is equal to the saturated density of the metal hydride bed ($\rho_{\text{sat}}$).
* Initial Temperature ($T(t=0)$):
    * *Absorption:* Initial temperature of the entire system (PCM and MHT tank) is set to 301.15 K.
    * *Desorption:* Initial temperature of the entire system is set to 305.15 K.
* Initial Hydrogen Pressure ($P(t=0)$): The initial hydrogen pressure is assumed to be equal to the equilibrium pressure at the initial temperature of the system. This equilibrium pressure is calculated using the Van't Hoff equation:
    $$
    \text{Ln}\left(\frac{P_{eq}}{P_{ref}}\right) = A - \frac{B}{T}
    $$

Where:
* $P_{eq}$: Equilibrium pressure of the MHT.
* $P_{ref}$: Reference pressure (1 MPa).
* $T$: Equilibrium temperature between hydrogen and the MHT.
* $A, B$: Constants specific to the absorption: A = 10.7 et B = 3704.6 / desorption: A = 10.57 et B = 3704.6 process.

For the initial condition of the simulation, the initial hydrogen pressure is assumed to be the equilibrium pressure at the initial system temperature ($T(t=0)$). We use the Van't Hoff equation with $T = T_{0}$ to calculate this initial pressure ($P_{initial} = P_{eq}(T_{initial})$).

Knowing the initial pressure and initial temperature, the initial density of the gaseous hydrogen ($\rho_g$) can then be determined using the ideal gas law : $\rho_{g, initial} = \frac{P_{initial}}{R_{specific} T_{initial}}$, with R the specific constante of hydrogen gaz : $R=4124$ J/mole K.


#### Boundary Conditions (BC)

* **Hydrogen Inlet ($z = -L_{in}$, $0 \leq x \leq R_{in}$):**
    * Pressure is defined as the supply pressure ($P_{\text{in}}$) - 10, 15 et 20 bar - for absorption or the discharge pressure ($P_{\text{out}}$) - 1.5, 1.75 et 2 bar - for desorption.
* **No Hydrogen Flux/Adiabatic Boundaries:**
    * *Vertical Wall ($z = 0$, $R_{in} \leq x \leq R$*)
        * Hydrogen velocities are zero: $u_x = 0, u_z = 0$.
        * No-slip condition at a solid, impermeable wall.
    * *Horizontal Wall ($x = R_{in}$, $-L_{in} \leq z \leq 0$ and $x = R$, $0 \leq z \leq L$):*
        * Hydrogen velocities are zero: $u_x = 0, u_z = 0$.
        * No-slip condition at solid, impermeable walls.
    * *Outlet ($z = L$, $0 \leq x \leq R$):*
        * Hydrogen velocities are zero: $u_x = 0, u_z = 0$.
        * Assuming no flow out of the defined domain at this boundary for this initial Navier-Stokes solve.
    * *Axis  ($x = 0$, $L_{in \leq z \leq L}$)*
        * $u_x = 0$, $\frac{\partial u_z}{\partial x} = 0$.
        * Symmetry condition representing the axis of the original cylindrical geometry, enforcing no flow across and symmetric tangential velocity.

### Weak Form of Incompressible Navier-Stokes Equations (2D)

The weak form of the 2D incompressible Navier-Stokes equations is derived by multiplying the strong form by appropriate test functions and integrating over the domain $\Omega$, followed by integration by parts (Green's theorem).

Let $\mathbf{v} = (v_x, v_z)$ be the vector test function for the momentum equation and $q$ be the scalar test function for the continuity equation.

**Equation 1:  Weak Form of Conservation of Mass Equation**

Multiply $\nabla \cdot \mathbf{u} = 0$ by $q$ and integrate over $\Omega$:
$$
\int_\Omega (\nabla \cdot \mathbf{u}) q \, d\Omega = 0
$$
Using the divergence theorem, the weak form is:
$$
\int_\Omega \mathbf{u} \cdot \nabla q \, d\Omega = \int_{\partial \Omega} (\mathbf{u} \cdot \mathbf{n}) q \, d\Gamma \quad \forall q
$$

**Equation 2: Weak Form of Momentum Equation:**

Multiply the momentum equation by $\mathbf{v}$ and integrate over $\Omega$:
$$
\int_\Omega \rho_g \left( \frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} \right) \cdot \mathbf{v} \, d\Omega = \int_\Omega (-\nabla p + \mu \nabla^2 \mathbf{u} + \mathbf{f}) \cdot \mathbf{v} \, d\Omega
$$
After integration by parts, the weak form becomes:
$$
\int_\Omega \rho_g \left( \frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u} \right) \cdot \mathbf{v} \, d\Omega + \int_\Omega \mu (\nabla \mathbf{u}) : (\nabla \mathbf{v}) \, d\Omega - \int_\Omega p (\nabla \cdot \mathbf{v}) \, d\Omega = \int_\Omega \mathbf{f} \cdot \mathbf{v} \, d\Omega + \int_{\partial \Omega} (\mu (\nabla \mathbf{u}) \mathbf{n} - p \mathbf{n}) \cdot \mathbf{v} \, d\Gamma \quad \forall \mathbf{v}
$$

<b>Finite Element Spatial Discretization</b>

<b>Time-Integration using DifferentialEquations.jl </b>

<b>Questions</b>:

## Section 2: Read 2D Mesh From External Input Mesh File  

In [4]:
grid = togrid("tankAndNozzle.msh")

Info    : Reading 'tankAndNozzle.msh'...
Info    : 15 entities
Info    : 2427 nodes
Info    : 2598 elements
Info    : Done reading 'tankAndNozzle.msh'


Grid{2, Quadrilateral, Float64} with 2254 Quadrilateral cells and 2427 nodes

## Section 3: Problem Set-Up (1/3)  
Set up parameters

In [16]:
# I. Spatial Parameters
L = 1. 
R = 0.1
Lin = -0.1*L 
Rin = 0.3*R 

# II. Time Parameters
t_0 = 0.0            # Initial time
t_end = 10.0         # End time of simulation
dt = 0.1             # Time step (use adaptive time stepping)

# III. Fluid Properties
# ρ_g will be calculated, so we don't initialize it here
μ = 8.4e-6           # Replace with the dynamic viscosity of hydrogen (in kg/(m s))
K_perm = 1e-9             # Permeability of the porous bed (m²)
R_specific = 4124.0  # Specific gas constant for hydrogen (J/kg·K)

# IV. Initial Conditions
ρ_s_initial = 0.0 # Placeholder
rhos_sat = 7259.0    # rho_saturé
rho_emp = 7164.0 # Empty bed density
T_absorption = 301.15  # K
T_desorption = 305.15  # K
# P_initial and P_eq will be calculated

# V. Van't Hoff Equation Parameters
P_ref = 1e6       # Reference pressure (1 MPa = 1e6 Pa)
A_absorption = 10.7
B_absorption = 3704.6
A_desorption = 10.57
B_desorption = 3704.6

# VI. Boundary Conditions
P_in_absorption_values = [10e5, 15e5, 20e5]  # Pa (converted from bar)
P_out_desorption_values = [1.5e5, 1.75e5, 2e5] # Pa (converted from bar)

# Function to calculate initial pressure (P_initial)
function calculate_P_eq(T, A, B, P_ref)
    return P_ref * exp(A - B / T)
end

# Calculation of ρ_g_initial
function calculate_rho_g_initial(P_initial, R_specific, T_initial)
    return P_initial / (R_specific * T_initial)
end

# Absorption case
T_init_abs = T_absorption # Choose which temperature to use
P_initial_abs = calculate_P_eq(T_init_abs, A_absorption, B_absorption, P_ref)
ρ_g_initial_abs = calculate_rho_g_initial(P_initial_abs, R_specific, T_init_abs)
ρ_g = ρ_g_initial_abs;

Set-up degrees-of-freedom handler (dh) and constraint handler (ch). 

In [17]:
dim = 2

# Interpolations quadrilatérales
ip_v = Lagrange{RefQuadrilateral, 2}()^dim
qr = QuadratureRule{RefQuadrilateral}(4)
cellvalues_v = CellValues(qr, ip_v);
println("Nombre de fonctions de base vues par cellvalues_v : ", getnbasefunctions(cellvalues_v))


ip_p = Lagrange{RefQuadrilateral, 1}()
cellvalues_p = CellValues(qr, ip_p);

dh = DofHandler(grid)
add!(dh, :v, ip_v)
add!(dh, :p, ip_p)
close!(dh);

ch = ConstraintHandler(dh);

# Boundary Conditions ----------------------------------------

# 1. No-slip boundaries (Vertical Wall, Horizontal Walls, Axis)
#    These are combined into a single no-slip condition.
noslip_facet_names = ["wall", "axis"];  # Corrected names from Gmsh
noslip_bc_facets = union(getfacetset.((grid, ), noslip_facet_names)...);
noslip_bc = Dirichlet(:v, noslip_bc_facets, (x, t) -> Vec((0.0, 0.0)), [1, 2]);
add!(ch, noslip_bc);

# 2. Hydrogen Inlet (Parabolic Inflow Profile)
inlet_facets = getfacetset(grid, "inlet");

function parabolic_inflow_profile(x, t)
    # Geometry parameters
    Lin = -0.1
    Rin = 0.03
    ymax = Rin  # The height of the inlet is Rin in the z-direction (assuming axis at z=0)
    
    # Time-dependent maximum velocity
    vmax = min(t * 1.5, 1.5)
    
    # Check if the point is on the inlet boundary (z = Lin, 0 <= x <= Rin)
    if abs(x[2] - Lin) < 1e-9 && x[1] >= 0 && x[1] <= Rin
        # Parabolic profile along the x-direction (from axis to Rin)
        # y in the original function corresponds to the direction perpendicular to the flow
        # Here, the flow is primarily in the z-direction, and the profile varies with x
        local r = x[1] # Radial coordinate (from axis)
        local R_inlet = Rin
        local vz_profile = 4 * vmax * (r / R_inlet) * (1 - (r / R_inlet))
        return Vec((0.0, vz_profile)) # Velocity in z-direction (component 2)
    else
        return Vec((0.0, 0.0))
    end
end
inflow_bc = Dirichlet(:v, inlet_facets, parabolic_inflow_profile, [1, 2]);
add!(ch, inflow_bc);

# 3. Outlet (Zero Velocity - Simplified from Pressure = 0 for Stokes with Dirichlet inlet)
outlet_facets = getfacetset(grid, "outlet");
outlet_bc = Dirichlet(:v, outlet_facets, (x, t) -> Vec((0.0, 0.0)), [1, 2]); # Zero velocity at outlet
add!(ch, outlet_bc);

close!(ch)
update!(ch, 0.0);

println("Corrected Ferrite configuration completed with parabolic inflow.");

Nombre de fonctions de base vues par cellvalues_v : 18
Corrected Ferrite configuration completed with parabolic inflow.


## Section 4: Problem Set-Up (2/3)
Assemble mass matrix (M) and Stokes matrix (K). Set up time interval. 

In [18]:
function assemble_mass_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, M::SparseMatrixCSC, dh::DofHandler, ρ_g)
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v_block = 1
    p_block = 2
    Mₑ = BlockArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    mass_assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Mₑ, 0)
        Ferrite.reinit!(cellvalues_v, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)
            for i in 1:n_basefuncs_v
                φᵢ = shape_value(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    φⱼ = shape_value(cellvalues_v, q_point, j)
                    Mₑ[BlockIndex((v_block, v_block), (i, j))] += ρ_g * (φᵢ ⋅ φⱼ) * dΩ
                end
            end
        end
        assemble!(mass_assembler, celldofs(cell), Mₑ)
    end
    return M
end

assemble_mass_matrix (generic function with 1 method)

In [19]:
function assemble_stiffness_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, μ, K_perm, K::SparseMatrixCSC, dh::DofHandler)
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v_dofs = 1:n_basefuncs_v
    p_dofs = (n_basefuncs_v + 1):n_basefuncs

    Kₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    stiffness_assembler = start_assemble(K)

    for cell in CellIterator(dh)
        fill!(Kₑ, 0)

        Ferrite.reinit!(cellvalues_v, cell)
        Ferrite.reinit!(cellvalues_p, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)

            # Block A: Viscous term + Porous media term (acting on velocity)
            for i in 1:n_basefuncs_v
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                φᵢ = shape_value(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    ∇φⱼ = shape_gradient(cellvalues_v, q_point, j)
                    φⱼ = shape_value(cellvalues_v, q_point, j)
                    Kₑ[BlockIndex((1, 1), (i, j))] += μ * (∇φᵢ ⊡ ∇φⱼ) * dΩ + (μ / K_perm) * (φᵢ ⋅ φⱼ) * dΩ
                end
            end

            # Block Bᵀ: Pressure term (acting on velocity equation)
            for i in 1:n_basefuncs_v
                divφᵢ = shape_divergence(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_p
                    ψⱼ = shape_value(cellvalues_p, q_point, j)
                    Kₑ[BlockIndex((1, 2), (i, j))] += -ψⱼ * divφᵢ * dΩ
                end
            end

            # Block B: Incompressibility term (acting on pressure equation)
            for j in 1:n_basefuncs_p
                ψⱼ = shape_value(cellvalues_p, q_point, j)
                for i in 1:n_basefuncs_v
                    divφᵢ = shape_divergence(cellvalues_v, q_point, i)
                    Kₑ[BlockIndex((2, 1), (j, i))] += ψⱼ * divφᵢ * dΩ
                end
            end

            # Block 0: Remains zero as there are no direct pressure-pressure terms in the momentum equation
            # Kₑ[BlockIndex((2, 2), :, :)] remains at its initial zero value
        end

        # Assemble `Kₑ` into the stiffness matrix `K`.
        assemble!(stiffness_assembler, celldofs(cell), Kₑ)
    end
    return K
end

assemble_stiffness_matrix (generic function with 1 method)

In [20]:
# Matrix allocation and assembly
M = allocate_matrix(dh);
M = assemble_mass_matrix(cellvalues_v, cellvalues_p, M, dh, ρ_g)

K = allocate_matrix(dh);
K = assemble_stiffness_matrix(cellvalues_v, cellvalues_p, μ, K_perm, K, dh);

u₀ = zeros(ndofs(dh))
apply!(u₀, ch);

jac_sparsity = sparse(K);

apply!(M, ch)

## Section 5:  Problem Set-Up (3/3), Time-Integration and Post-Processing 

In [25]:
struct RHSparams
    K::SparseMatrixCSC
    ch::ConstraintHandler
    dh::DofHandler
    cellvalues_v::CellValues
    u::Vector
end
p = RHSparams(K, ch, dh, cellvalues_v, copy(u₀))

function ferrite_limiter!(u, _, p, t)
    update!(p.ch, t)
    apply!(u, p.ch)
end

function navierstokes_rhs_element!(dvₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            dvₑ[j] -= v ⋅ ∇v' ⋅ φⱼ * dΩ
        end
    end
end

function navierstokes!(du,u_uc,p::RHSparams,t)

    @unpack K,ch,dh,cellvalues_v,u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    mul!(du, K, u) # du .= K * u

    # nonlinear contribution
    v_range = dof_range(dh, :v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    vₑ = zeros(n_basefuncs)
    duₑ = zeros(n_basefuncs)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]
        vₑ .= @views u[v_celldofs]
        fill!(duₑ, 0.0)
        navierstokes_rhs_element!(duₑ, vₑ, cellvalues_v)
        assemble!(du, v_celldofs, duₑ)
    end
end;

function navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues_v, q_point, i)
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                Jₑ[j, i] -= (φᵢ ⋅ ∇v' + v ⋅ ∇φᵢ') ⋅ φⱼ * dΩ
            end
        end
    end
end

function navierstokes_jac!(J,u_uc,p,t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    # Here we assume that J has exactly the same structure as K by construction
    nonzeros(J) .= nonzeros(K)

    assembler = start_assemble(J; fillzero=false)

    # Assemble variation of the nonlinear term
    n_basefuncs = getnbasefunctions(cellvalues_v)
    Jₑ = zeros(n_basefuncs, n_basefuncs)
    vₑ = zeros(n_basefuncs)
    v_range = dof_range(dh, :v)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]

        vₑ .= @views u[v_celldofs]
        fill!(Jₑ, 0.0)
        navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
        assemble!(assembler, v_celldofs, Jₑ)
    end

    apply!(J, ch)
end;

rhs = ODEFunction(navierstokes!, mass_matrix=M; jac=navierstokes_jac!, jac_prototype=jac_sparsity)
problem = ODEProblem(rhs, u₀, (0.0,T), p);

struct FreeDofErrorNorm
    ch::ConstraintHandler
end
(fe_norm::FreeDofErrorNorm)(u::Union{AbstractFloat, Complex}, t) = DiffEqBase.ODE_DEFAULT_NORM(u, t)
(fe_norm::FreeDofErrorNorm)(u::AbstractArray, t) = DiffEqBase.ODE_DEFAULT_NORM(u[fe_norm.ch.free_dofs], t)

#timestepper = Rodas5P(autodiff=false, step_limiter! = ferrite_limiter!);

integrator = init(
    #problem, timestepper; initializealg=NoInit(), dt=Δt₀,
    problem; initializealg=NoInit(), dt=Δt₀,
    adaptive=true, abstol=1e-4, reltol=1e-5,
    progress=true, progress_steps=1,
    verbose=true, internalnorm=FreeDofErrorNorm(ch), d_discontinuities=[1.0]
);

LoadError: UndefVarError: `u₀` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [26]:
pvd = paraview_collection("vortex-street")
for (step, (u,t)) in enumerate(intervals(integrator))
    VTKGridFile("vortex-street-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
end
vtk_save(pvd);

LoadError: UndefVarError: `integrator` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Section 6: Full Code From Ferrite Here Intended as Reference

In [ ]:
function assemble_mass_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, M::SparseMatrixCSC, dh::DofHandler)
    # Allocate a buffer for the local matrix and some helpers, together with the assembler.
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Mₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # It follows the assembly loop as explained in the basic tutorials.
    mass_assembler = start_assemble(M)
    for cell in CellIterator(dh)
        fill!(Mₑ, 0)
        Ferrite.reinit!(cellvalues_v, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)
            # Remember that we assemble a vector mass term, hence the dot product.
            # There is only one time derivative on the left hand side, so only one mass block is non-zero.
            for i in 1:n_basefuncs_v
                φᵢ = shape_value(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    φⱼ = shape_value(cellvalues_v, q_point, j)
                    Mₑ[BlockIndex((v▄, v▄), (i, j))] += φᵢ ⋅ φⱼ * dΩ
                end
            end
        end
        assemble!(mass_assembler, celldofs(cell), Mₑ)
    end

    return M
end;

function assemble_stokes_matrix(cellvalues_v::CellValues, cellvalues_p::CellValues, ν, K::SparseMatrixCSC, dh::DofHandler)
    # Again, some buffers and helpers
    n_basefuncs_v = getnbasefunctions(cellvalues_v)
    n_basefuncs_p = getnbasefunctions(cellvalues_p)
    n_basefuncs = n_basefuncs_v + n_basefuncs_p
    v▄, p▄ = 1, 2
    Kₑ = BlockedArray(zeros(n_basefuncs, n_basefuncs), [n_basefuncs_v, n_basefuncs_p], [n_basefuncs_v, n_basefuncs_p])

    # Assembly loop
    stiffness_assembler = start_assemble(K)
    for cell in CellIterator(dh)
        # Don't forget to initialize everything
        fill!(Kₑ, 0)

        Ferrite.reinit!(cellvalues_v, cell)
        Ferrite.reinit!(cellvalues_p, cell)

        for q_point in 1:getnquadpoints(cellvalues_v)
            dΩ = getdetJdV(cellvalues_v, q_point)

            for i in 1:n_basefuncs_v
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                for j in 1:n_basefuncs_v
                    ∇φⱼ = shape_gradient(cellvalues_v, q_point, j)
                    Kₑ[BlockIndex((v▄, v▄), (i, j))] -= ν * ∇φᵢ ⊡ ∇φⱼ * dΩ
                end
            end

            for j in 1:n_basefuncs_p
                ψ = shape_value(cellvalues_p, q_point, j)
                for i in 1:n_basefuncs_v
                    divφ = shape_divergence(cellvalues_v, q_point, i)
                    Kₑ[BlockIndex((v▄, p▄), (i, j))] += (divφ * ψ) * dΩ
                    Kₑ[BlockIndex((p▄, v▄), (j, i))] += (ψ * divφ) * dΩ
                end
            end
        end

        # Assemble `Kₑ` into the Stokes matrix `K`.
        assemble!(stiffness_assembler, celldofs(cell), Kₑ)
    end
    return K
end;

T = 6.0
Δt₀ = 0.001
Δt_save = 0.1

M = allocate_matrix(dh);
M = assemble_mass_matrix(cellvalues_v, cellvalues_p, M, dh);

K = allocate_matrix(dh);
K = assemble_stokes_matrix(cellvalues_v, cellvalues_p, ν, K, dh);

u₀ = zeros(ndofs(dh))
apply!(u₀, ch);

jac_sparsity = sparse(K);

apply!(M, ch)

struct RHSparams
    K::SparseMatrixCSC
    ch::ConstraintHandler
    dh::DofHandler
    cellvalues_v::CellValues
    u::Vector
end
p = RHSparams(K, ch, dh, cellvalues_v, copy(u₀))

function ferrite_limiter!(u, _, p, t)
    update!(p.ch, t)
    apply!(u, p.ch)
end

function navierstokes_rhs_element!(dvₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            dvₑ[j] -= v ⋅ ∇v' ⋅ φⱼ * dΩ
        end
    end
end

function navierstokes!(du,u_uc,p::RHSparams,t)

    @unpack K,ch,dh,cellvalues_v,u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    mul!(du, K, u) # du .= K * u

    # nonlinear contribution
    v_range = dof_range(dh, :v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    vₑ = zeros(n_basefuncs)
    duₑ = zeros(n_basefuncs)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]
        vₑ .= @views u[v_celldofs]
        fill!(duₑ, 0.0)
        navierstokes_rhs_element!(duₑ, vₑ, cellvalues_v)
        assemble!(du, v_celldofs, duₑ)
    end
end;

function navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
    n_basefuncs = getnbasefunctions(cellvalues_v)
    for q_point in 1:getnquadpoints(cellvalues_v)
        dΩ = getdetJdV(cellvalues_v, q_point)
        ∇v = function_gradient(cellvalues_v, q_point, vₑ)
        v = function_value(cellvalues_v, q_point, vₑ)
        for j in 1:n_basefuncs
            φⱼ = shape_value(cellvalues_v, q_point, j)

            for i in 1:n_basefuncs
                φᵢ = shape_value(cellvalues_v, q_point, i)
                ∇φᵢ = shape_gradient(cellvalues_v, q_point, i)
                Jₑ[j, i] -= (φᵢ ⋅ ∇v' + v ⋅ ∇φᵢ') ⋅ φⱼ * dΩ
            end
        end
    end
end

function navierstokes_jac!(J,u_uc,p,t)

    @unpack K, ch, dh, cellvalues_v, u = p

    u .= u_uc
    update!(ch, t)
    apply!(u, ch)

    # Linear contribution (Stokes operator)
    # Here we assume that J has exactly the same structure as K by construction
    nonzeros(J) .= nonzeros(K)

    assembler = start_assemble(J; fillzero=false)

    # Assemble variation of the nonlinear term
    n_basefuncs = getnbasefunctions(cellvalues_v)
    Jₑ = zeros(n_basefuncs, n_basefuncs)
    vₑ = zeros(n_basefuncs)
    v_range = dof_range(dh, :v)
    for cell in CellIterator(dh)
        Ferrite.reinit!(cellvalues_v, cell)
        v_celldofs = @view celldofs(cell)[v_range]

        vₑ .= @views u[v_celldofs]
        fill!(Jₑ, 0.0)
        navierstokes_jac_element!(Jₑ, vₑ, cellvalues_v)
        assemble!(assembler, v_celldofs, Jₑ)
    end

    apply!(J, ch)
end;

rhs = ODEFunction(navierstokes!, mass_matrix=M; jac=navierstokes_jac!, jac_prototype=jac_sparsity)
problem = ODEProblem(rhs, u₀, (0.0,T), p);

struct FreeDofErrorNorm
    ch::ConstraintHandler
end
(fe_norm::FreeDofErrorNorm)(u::Union{AbstractFloat, Complex}, t) = DiffEqBase.ODE_DEFAULT_NORM(u, t)
(fe_norm::FreeDofErrorNorm)(u::AbstractArray, t) = DiffEqBase.ODE_DEFAULT_NORM(u[fe_norm.ch.free_dofs], t)

timestepper = Rodas5P(autodiff=false, step_limiter! = ferrite_limiter!);

integrator = init(
    problem, timestepper; initializealg=NoInit(), dt=Δt₀,
    adaptive=true, abstol=1e-4, reltol=1e-5,
    progress=true, progress_steps=1,
    verbose=true, internalnorm=FreeDofErrorNorm(ch), d_discontinuities=[1.0]
);

pvd = paraview_collection("vortex-street")
for (step, (u,t)) in enumerate(intervals(integrator))
    VTKGridFile("vortex-street-$step", dh) do vtk
        write_solution(vtk, dh, u)
        pvd[t] = vtk
    end
end
vtk_save(pvd);

### Whole Formulation of the problem 

**Equation 3: Solid Density Equation**
$$
(1 - \epsilon) \frac{\partial \rho_s}{\partial t} = \dot{m}(\rho_s, t)
$$

$$
\epsilon \frac{\partial \rho_g}{\partial t} = D \left( \frac{\partial^2 \rho_g}{\partial x^2} + \frac{\partial^2 \rho_g}{\partial z^2} \right) + u_x \frac{\partial \rho_g}{\partial x} + u_z \frac{\partial \rho_g}{\partial z} + \dot{m}(\rho_s, t)
$$

**Equation 4: Conservation of energy :Temperature Equation for the Metal Hydride Tank (MHT) and Gaseous Hydrogen**

$$ \left( \rho C_p \right)_{\text{eff}} \frac{\partial T}{\partial t} + \left( \varepsilon \rho_g C_{p,g} u_x \frac{\partial T}{\partial x} \right) + \left( \varepsilon \rho_g C_{p,g} u_z \frac{\partial T}{\partial z} \right) = \frac{\partial}{\partial x} \left( k_{\text{eff}} \frac{\partial T}{\partial x} \right) + \frac{\partial}{\partial z} \left( k_{\text{eff}} \frac{\partial T}{\partial z} \right) - \dot{m} \left( (1-\varepsilon) \Delta H + T (C_{p,g} - C_s) \right) $$

* *Terms Breakdown (adapted for 2D Cartesian):*
    * $\left( \rho C_p \right)_{\text{eff}} \frac{\partial T}{\partial t}$: Transient term.
    * $\left( \varepsilon \rho_g C_{p,g} u_x \frac{\partial T}{\partial x} \right) + \left( \varepsilon \rho_g C_{p,g} u_z \frac{\partial T}{\partial z} \right)$: Convective heat transfer in $x$ and $z$ directions ($u_x, u_z$ are velocity components in $x$ and $z$).
    * $\frac{\partial}{\partial x} \left( k_{\text{eff}} \frac{\partial T}{\partial x} \right) + \frac{\partial}{\partial z} \left( k_{\text{eff}} \frac{\partial T}{\partial z} \right)$: Conductive heat transfer in $x$ and $z$ directions.
    * $-\dot{m} \left( (1-\varepsilon) \Delta H + T (C_{p,g} - C_s) \right)$: Source term due to reaction.

* *Effective Properties (unchanged):*
    * $(\rho C_p)_{\text{eff}} = \varepsilon \rho_g C_{p,g} + (1-\varepsilon) \rho_s C_s$
    * $k_{\text{eff}} = \varepsilon k_g + (1-\varepsilon) k_s$

---

#### Initial Conditions (IC)

* Initial Solid Density ($\rho_s(t=0)$):
    * *Absorption:* Initial solid density is equal to the density of the metal hydride bed without hydrogen ($\rho_{\text{emp}}$).
    * *Desorption:* Initial solid density is equal to the saturated density of the metal hydride bed ($\rho_{\text{sat}}$).
* Initial Temperature ($T(t=0)$):
    * *Absorption:* Initial temperature of the entire system (PCM and MHT tank) is set to 301.15 K.
    * *Desorption:* Initial temperature of the entire system is set to 305.15 K.
* Initial Hydrogen Pressure ($P(t=0)$): The initial hydrogen pressure is assumed to be equal to the equilibrium pressure at the initial temperature of the system. This equilibrium pressure is calculated using the Van't Hoff equation:
    $$
    \text{Ln}\left(\frac{P_{eq}}{P_{ref}}\right) = A - \frac{B}{T}
    $$

Where:
* $P_{eq}$: Equilibrium pressure of the MHT.
* $P_{ref}$: Reference pressure (1 MPa).
* $T$: Equilibrium temperature between hydrogen and the MHT.
* $A, B$: Constants specific to the absorption: A = 10.7 et B = 3704.6 / desorption: A = 10.57 et B = 3704.6 process.

For the initial condition of the simulation, the initial hydrogen pressure is assumed to be the equilibrium pressure at the initial system temperature ($T(t=0)$). We use the Van't Hoff equation with $T = T_{0}$ to calculate this initial pressure ($P_{initial} = P_{eq}(T_{initial})$).

Knowing the initial pressure and initial temperature, the initial density of the gaseous hydrogen ($\rho_g$) can then be determined using the ideal gas law (as implemented in software like Fluent, according to Darzi's thesis).

#### Boundary Conditions (BC)

* **Hydrogen Inlet ($z = -L_{in}$, $0 \leq x \leq R_{in}$):**
    * Pressure is defined as the supply pressure ($P_{\text{in}}$) - 10, 15 et 20 bar - for absorption or the discharge pressure ($P_{\text{out}}$) - 1.5, 1.75 et 2 bar - for desorption.
* **No Hydrogen Flux/Adiabatic Boundaries:**
    * *Vertical Wall ($z = 0$, $R_{in} \leq x \leq R$*)
        * Axial temperature gradient is zero: $\frac{\partial T}{\partial z} = 0$.
        * Hydrogen velocities are zero: $u_x = 0, u_z = 0$.
    * *Horizontal dWall ($x = R_{in}$, $-L_{in} \leq z \leq 0$ and ($x = R$, $0 \leq z \leq L$):*
        * Temperature gradient is zero: $\frac{\partial T}{\partial x} = 0$ (adiabatic outer wall).
        * Hydrogen velocities are zero: $u_x = 0, u_z = 0$.
    * *Outlet ($z = L$, $0 \leq x \leq R$):*
        * Axial temperature gradient is zero: $\frac{\partial T}{\partial z} = 0$.
        * Hydrogen velocities are zero: $u_x = 0, u_z = 0$.
    * What about the axis : neuman condition 